#  Параметрические критерии 

In [1]:
import numpy as np
import pandas as pd

from scipy import stats
from statsmodels.stats.proportion import proportion_confint
from statsmodels.stats.weightstats import CompareMeans, DescrStatsW, ztest

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

В одном из выпусков программы "Разрушители легенд" проверялось, действительно ли заразительна зевота. В эксперименте участвовало 50 испытуемых, проходивших собеседование на программу. Каждый из них разговаривал с рекрутером; в конце 34 из 50 бесед рекрутер зевал. Затем испытуемых просили подождать решения рекрутера в соседней пустой комнате.

Во время ожидания 10 из 34 испытуемых экспериментальной группы и 4 из 16 испытуемых контрольной начали зевать. Таким образом, разница в доле зевающих людей в этих двух группах составила примерно 4.4%. Ведущие заключили, что миф о заразительности зевоты подтверждён.

Можно ли утверждать, что доли зевающих в контрольной и экспериментальной группах отличаются статистически значимо? Посчитайте достигаемый уровень значимости при альтернативе заразительности зевоты, округлите до четырёх знаков после десятичной точки.

In [2]:
n = 50
n1 = 34
n2 = 16
a = 10
b = 4

In [3]:
p1 = a/n1
p2 = b/n2
P = (p1*n1 + p2*n2) / (n1 + n2)

In [4]:
print(P)

0.28


In [5]:
stat_Z=(p1 - p2) / np.sqrt(P * (1 - P) * (1. / n1 + 1. / n2))

In [6]:
print(stat_Z)

0.32410186177608225


In [7]:
def proportions_diff_z_test(z_stat, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    if alternative == 'two-sided':
        return 2 * (1 - stats.norm.cdf(np.abs(z_stat)))
    
    if alternative == 'less':
        return stats.norm.cdf(z_stat)

    if alternative == 'greater':
        return 1 - stats.norm.cdf(z_stat)

In [8]:
print("two-sided: %.4f; less: %.4f; greater: %.4f" % (proportions_diff_z_test(stat_Z, 'two-sided'), proportions_diff_z_test(stat_Z, 'less'), proportions_diff_z_test(stat_Z, 'greater')))

two-sided: 0.7459; less: 0.6271; greater: 0.3729


Имеются данные измерений двухсот швейцарских тысячефранковых банкнот, бывших в обращении в первой половине XX века. Сто из банкнот были настоящими, и сто — поддельными. На рисунке ниже показаны измеренные признаки:

Отделите 50 случайных наблюдений в тестовую выборку с помощью функции sklearn.cross_validation.train_test_split (зафиксируйте random state = 1). На оставшихся 150 настройте два классификатора поддельности банкнот:

    логистическая регрессия по признакам X1​,X2​,X3​;
    логистическая регрессия по признакам X4​,X5​,X6​.

Каждым из классификаторов сделайте предсказания меток классов на тестовой выборке. Одинаковы ли доли ошибочных предсказаний двух классификаторов? Проверьте гипотезу, вычислите достигаемый уровень значимости. Введите номер первой значащей цифры (например, если вы получили $ 5.5×10−85.5\times10^{-8}5.5×10−8 $, нужно ввести 8).

In [9]:
data = pd.read_table('banknotes.txt')

In [10]:
data.head()

,X1,X2,X3,X4,X5,X6,real
0,214.8,131.0,131.1,9.0,9.7,141.0,1
1,214.6,129.7,129.7,8.1,9.5,141.7,1
2,214.8,129.7,129.7,8.7,9.6,142.2,1
3,214.8,129.7,129.6,7.5,10.4,142.0,1
4,215.0,129.6,129.7,10.4,7.7,141.8,1


In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 7 columns):
X1      200 non-null float64
X2      200 non-null float64
X3      200 non-null float64
X4      200 non-null float64
X5      200 non-null float64
X6      200 non-null float64
real    200 non-null int64
dtypes: float64(6), int64(1)
memory usage: 11.1 KB


In [12]:
data.describe()

,X1,X2,X3,X4,X5,X6,real
count,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000
mean,214.896000,130.121500,129.956500,9.417500,10.650500,140.483500,0.500000
std,0.376554,0.361026,0.404072,1.444603,0.802947,1.152266,0.501255
min,213.800000,129.000000,129.000000,7.200000,7.700000,137.800000,0.000000
25%,214.600000,129.900000,129.700000,8.200000,10.100000,139.500000,0.000000
50%,214.900000,130.200000,130.000000,9.100000,10.600000,140.450000,0.500000
75%,215.100000,130.400000,130.225000,10.600000,11.200000,141.500000,1.000000
max,216.300000,131.000000,131.100000,12.700000,12.300000,142.400000,1.000000


In [13]:
data.columns

Index(['X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'real'], dtype='object')

In [14]:
X = data.drop('real', axis=1)
y = data['real']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=50, random_state=1)

print(X.shape, y.shape, sep='\n')

print('Train shape: ', X_train.shape)
print('Test shape: ', X_test.shape)

(200, 6)
(200,)
Train shape:  (150, 6)
Test shape:  (50, 6)


In [15]:
features_1 = ['X1', 'X2', 'X3']
features_2 = ['X4', 'X5', 'X6']

In [16]:
import warnings
warnings.filterwarnings('ignore')

In [17]:
linear_clf_1 = LogisticRegression()
linear_clf_1.fit(X_train[features_1], y_train)
predictions_1 = linear_clf_1.predict(X_test[features_1])
accuracy_features1 = accuracy_score(y_test, predictions_1)
print(accuracy_features1)

0.8


In [18]:
linear_clf_2 = LogisticRegression()
linear_clf_2.fit(X_train[features_2], y_train)
predictions_2 = linear_clf_2.predict(X_test[features_2])
accuracy_features2 = accuracy_score(y_test, predictions_2)
print(accuracy_features2)

0.98


In [19]:
from statsmodels.stats.weightstats import *

In [20]:
stats.ttest_rel(predictions_1, predictions_2)

Ttest_relResult(statistic=-1.5275252316519468, pvalue=0.13306042398416879)

In [21]:
def proportions_diff_confint_rel(sample1, sample2, alpha = 0.05):
    z = stats.norm.ppf(1 - alpha / 2.)
    sample = list(zip(sample1, sample2))
    n = len(sample)
        
    f = sum([1 if (x[0] == 1 and x[1] == 0) else 0 for x in sample])
    g = sum([1 if (x[0] == 0 and x[1] == 1) else 0 for x in sample])
    
    left_boundary = float(f - g) / n  - z * np.sqrt(float((f + g)) / n**2 - float((f - g)**2) / n**3)
    right_boundary = float(f - g) / n  + z * np.sqrt(float((f + g)) / n**2 - float((f - g)**2) / n**3)
    return (left_boundary, right_boundary)

In [22]:
def proportions_diff_z_stat_rel(sample1, sample2):
    sample = list(zip(sample1, sample2))
    n = len(sample)
    
    f = sum([1 if (x[0] == 1 and x[1] == 0) else 0 for x in sample])
    g = sum([1 if (x[0] == 0 and x[1] == 1) else 0 for x in sample])
    
    return float(f - g) / np.sqrt(f + g - float((f - g)**2) / n )

In [23]:
print('95%% confidence interval for a difference between predictions: [%.4f, %.4f]' %
      proportions_diff_confint_rel(predictions_1, predictions_2))

95% confidence interval for a difference between predictions: [-0.2270, 0.0270]


In [24]:
print('p-value: %f' % proportions_diff_z_test(proportions_diff_z_stat_rel(predictions_1, predictions_2)))

p-value: 0.122823


Ежегодно более 200000 людей по всему миру сдают стандартизированный экзамен GMAT при поступлении на программы MBA. Средний результат составляет 525 баллов, стандартное отклонение — 100 баллов.

Сто студентов закончили специальные подготовительные курсы и сдали экзамен. Средний полученный ими балл — 541.4. Проверьте гипотезу о неэффективности программы против односторонней альтернативы о том, что программа работает. Отвергается ли на уровне значимости 0.05 нулевая гипотеза? Введите достигаемый уровень значимости, округлённый до 4 знаков после десятичной точки.

In [25]:
def z_test(mean_val, exp_val, st_dev, num):
    standard_error = st_dev / np.sqrt(num)
    return (mean_val - exp_val) / standard_error

In [26]:
def p_val_greater(z_stat):
    return 1 - stats.norm.cdf(z_stat)

In [27]:
avg = 541.4
mu = 525
avg_sample = np.ones(100) * avg
st_dev = 100 

In [28]:
z_stat = z_test(avg, mu, st_dev, (len(avg_sample)))
z_stat

1.6399999999999977

In [29]:
p_val = p_val_greater(z_stat)
p_val

0.05050258347410397

Оцените теперь эффективность подготовительных курсов, средний балл 100 выпускников которых равен 541.5. Отвергается ли на уровне значимости 0.05 та же самая нулевая гипотеза против той же самой альтернативы? Введите достигаемый уровень значимости, округлённый до 4 знаков после десятичной точки. 

In [30]:
avg_1 = 541.5

In [31]:
z_stat1 = z_test(avg_1, mu, st_dev, (len(avg_sample)))
z_stat1

1.65

In [32]:
p_val1 = p_val_greater(z_stat1)
p_val1

0.0494714680336481